# Selección de Variables: Regresión Múltiple y el Efecto "Eclipse"

Cuando construimos un modelo, no basta con saber que una variable tiene una correlación alta. Necesitamos saber si esa variable aporta **información única** o si solo está "reflejando" el efecto de otra.

## 1. El Concepto de "Pesos Ajustados" (Coeficientes Beta)

En una regresión simple ($y = \beta_0 + \beta_1 x$), el coeficiente nos dice cuánto cambia $y$ por cada unidad de $x$. 

Pero en una **Regresión Múltiple**:
$$y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + ... + \beta_n x_n$$

Los coeficientes $\beta$ se convierten en **pesos ajustados**. Esto significa que $\beta_1$ representa el efecto de $x_1$ sobre $y$ **manteniendo todas las demás variables constantes**. Es como si "limpiáramos" el efecto de las otras variables para ver el impacto puro de $x_1$.

---

## 2. El Fenómeno del "Eclipse" (Multicolinealidad)

A veces, una variable que parecía muy importante en una correlación simple, "desaparece" o se vuelve insignificante en una regresión múltiple. Esto sucede por dos razones:

1. **Redundancia:** La variable no aporta nada nuevo que otra no haya explicado ya.
2. **Mediación/Eclipsamiento:** Una variable "X" en realidad solo afecta a "Y" a través de una variable "Z". Cuando incluyes a "Z" en el modelo, "X" se queda sin "luz" (se eclipsa).

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt

# --- SIMULACIÓN DIDÁCTICA ---
np.random.seed(42)
n = 100

# Variable A: Horas de estudio (Influencia real)
horas_estudio = np.random.normal(10, 2, n)

# Variable B: Dinero gastado en café (Altamente correlacionada con horas de estudio, 
# pero no mejora la nota por sí sola)
cafe = horas_estudio + np.random.normal(0, 0.5, n)

# Variable C: Inteligencia/Capacidad previa (Influencia real)
capacidad = np.random.normal(100, 15, n)

# Variable Objetivo: Nota del examen
# Nota = 2*(horas) + 0.1*(capacidad) + ruido (El café NO está en la fórmula real)
nota = (2 * horas_estudio) + (0.05 * capacidad) + np.random.normal(0, 2, n)

df = pd.DataFrame({'Nota': nota, 'Estudio': horas_estudio, 'Café': cafe, 'Capacidad': capacidad})

# 1. Veamos la correlación simple
print("Correlación Simple con la Nota:")
print(df.corr()['Nota'])

Correlación Simple con la Nota:
Nota         1.000000
Estudio      0.873792
Café         0.838317
Capacidad    0.377399
Name: Nota, dtype: float64


## 3. Detectando el Eclipsamiento con el Modelo

Si miras la correlación de arriba, el **Café** parece tener una relación fuerte con la **Nota**. Un analista novato diría: "¡Beber café sube las notas!". 

Vamos a ajustar el modelo múltiple para ver si el Café sobrevive cuando "controlamos" por las Horas de Estudio.

In [2]:
# Definimos X (predictores) y agregamos una constante (el intercepto)
X = df[['Estudio', 'Café', 'Capacidad']]
X = sm.add_constant(X)
y = df['Nota']

# Ajustamos el modelo
modelo = sm.OLS(y, X).fit()

print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:                   Nota   R-squared:                       0.810
Model:                            OLS   Adj. R-squared:                  0.804
Method:                 Least Squares   F-statistic:                     136.3
Date:                Wed, 31 Dec 2025   Prob (F-statistic):           1.75e-34
Time:                        18:58:41   Log-Likelihood:                -196.78
No. Observations:                 100   AIC:                             401.6
Df Residuals:                      96   BIC:                             412.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.5896      1.359      1.170      0.2

## 4. Cómo interpretar los resultados para decidir qué variable dejar

Para saber si una variable influye verdaderamente o está eclipsada, miramos tres cosas en la tabla de resultados anterior:

1. **P-valor del Coeficiente ($P > |t|$):** * Si es $< 0.05$, la variable aporta información **única** que las demás no tienen.
   * En nuestro ejemplo, verás que el p-valor del **Café** sube mucho (deja de ser significativo), mientras que el de **Estudio** se mantiene bajo. El Café ha sido "eclipsado".

2. **VIF (Factor de Inflación de la Varianza):**
   * Es la herramienta técnica para detectar el "eclipse". 
   * Si el VIF es $> 5$ o $10$, hay demasiada redundancia entre variables.

3. **Cambio en los Coeficientes:**
   * Si el coeficiente de una variable cambia drásticamente de signo o cae a casi cero al añadir otra variable, has detectado una variable "eclipsadora".

In [3]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Calculamos el VIF para cada variable
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print("\n--- Análisis de Redundancia (VIF) ---")
print(vif_data[vif_data['Variable'] != 'const'])


--- Análisis de Redundancia (VIF) ---
    Variable        VIF
1    Estudio  14.799920
2       Café  14.746528
3  Capacidad   1.037920


### **Resumen Didáctico**

1. La Correlación es como ver a dos personas caminando juntas; no sabes quién guía a quién.

2. La Regresión Múltiple es como pedirle a una persona que se detenga para ver si la otra sigue caminando hacia el mismo lugar.

3. Si una variable tiene un VIF alto y un P-valor alto en presencia de otras, es una candidata perfecta para ser eliminada: está eclipsada y solo añade ruido al modelo.